In [1]:
# %%
# create unique storm label for spatially connected regions where precip>0. 
# use class/patch_stormid_acrossmonths.py to patch storm ids across months
import xarray as xr
import numpy as np
import pandas as pd
import skimage.measure
from skimage.morphology import remove_small_objects
import pyarrow.feather as feather
from dask.distributed import Client
import shutil 
import os
import glob
# %%
def find_storms(mrms_ds):
    # create binary ndarray
    a = xr.where(mrms_ds > 0, 1, 0)
    # label objects
    storm_id, count_storms = skimage.measure.label(a, connectivity=1, return_num=True)
    # remove object SMALLER than 5 px
    #storm_id = remove_small_objects(storm_id, 100)
    
    return storm_id

def storm_properties(storm_id):
    object_features = skimage.measure.regionprops(storm_id)
    #area = [objf["area"] for objf in object_features]
    #axis_major_length = [objf["axis_major_length"] for objf in object_features]
    #axis_minor_length = [objf["axis_minor_length"] for objf in object_features]
    centroid = [objf["centroid"] for objf in object_features]
    # round centroid values to index xarray
    centroid = [[round(i[0]),round(i[1]),round(i[2])] for i in centroid]
    # flatten array to get eccentricity??
    #eccentricity = [objf["eccentricity"] for objf in object_features]

    return centroid



In [2]:
# Create a path to the code file
data_folder = os.path.join('..', '..','..','MRMS','2min_rate_cat_month_CO')
filenames = glob.glob(data_folder+'//'+'*.grib2')

destination = os.path.join('..', '..','..','storm_stats')

#client = Client()

In [3]:
client

NameError: name 'client' is not defined

In [4]:

# %%
for i in range(len(filenames)):
    print(i)

    month = xr.open_dataset(filenames[i], engine = "cfgrib",chunks={'time': '500MB'})

    month = month.where(month.longitude<=256,drop=True)
    #month = month.unknown.where(month.unknown > 0)
    month = month.unknown

    print(month)
    storm_id = find_storms(month)
    centroid = storm_properties(storm_id)

    data = {'centroid_index':centroid}

    storm_param = pd.DataFrame(data = data)

    name = filenames[i][0:8]+'_regionprops'
    storm_param.to_feather(destination+name)

    storm_id = storm_id.astype('float32')
    # create dataset from storm_id
    time = month.time
    latitude = month.latitude 
    longitude = month.longitude

    ds = xr.Dataset(data_vars=dict(storm_id=(["time", "latitude", "longitude"], storm_id),),
        coords=dict(time=time,latitude=latitude,longitude=longitude,))

    name = filenames[i][0:8]+'storm_id'

    path = destination+name+'.nc'
    ds.to_netcdf(path=path)


# %%


0


Ignoring index file '..\\..\\..\\MRMS\\2min_rate_cat_month_CO\\2018_aug_rate_CO.grib2.923a8.idx' older than GRIB file
C:\ProgramData\miniconda3\envs\radar\lib\site-packages\xarray\core\indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.DataArray 'unknown' (time: 22257, latitude: 400, longitude: 505)>
dask.array<where, shape=(22257, 400, 505), dtype=float32, chunksize=(443, 400, 505), chunktype=numpy.ndarray>
Coordinates:
  * time            (time) datetime64[ns] 2018-08-01 ... 2018-08-31T23:58:00
    step            timedelta64[ns] 00:00:00
    heightAboveSea  float64 0.0
  * latitude        (latitude) float64 37.01 37.02 37.02 ... 40.97 40.98 40.99
  * longitude       (longitude) float64 251.0 251.0 251.0 ... 256.0 256.0 256.0
    valid_time      (time) datetime64[ns] dask.array<chunksize=(22257,), meta=np.ndarray>
Attributes: (12/29)
    GRIB_paramId:                             0
    GRIB_dataType:                            ra
    GRIB_numberOfPoints:                      281600
    GRIB_typeOfLevel:                         heightAboveSea
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:   

MemoryError: Unable to allocate 476. MiB for an array with shape (443, 400, 704) and data type float32